<a href="https://colab.research.google.com/github/Sibusisongwenya/WIP-Project/blob/main/tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# utils/tuning.py

import itertools
import logging
import time
import numpy as np
import pandas as pd
import torch

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


def tune_hyperparameters(train_loader, val_loader, param_grids, tuning_method, bayesian_model_class, output_dir, class_weighted_loss, device):
    """
    Performs hyperparameter tuning for Bayesian models.

    Args:
        train_loader: Training DataLoader.
        val_loader: Validation DataLoader.
        param_grids (dict): Dictionary of parameters to try.
        tuning_method (str): 'grid' for exhaustive grid search or 'random' for random search.
        bayesian_model_class: The Bayesian model class to instantiate.
        output_dir (str): Directory to save tuning results.
        class_weighted_loss (bool): Whether to use class weighted loss.
        device (torch.device): The computation device.

    Returns:
        dict: The best configuration found.
    """
    logging.info("Starting hyperparameter tuning...")
    results = []
    start_time = time.time()

    # Generate parameter combinations
    if tuning_method == 'grid':
        keys = list(param_grids.keys())
        combinations = list(itertools.product(*param_grids.values()))
        param_combinations = [dict(zip(keys, comb)) for comb in combinations]
    else:
        # Random search: sample N times (N=5 for example)
        N = 5
        param_combinations = []
        for _ in range(N):
            config = {key: np.random.choice(values) for key, values in param_grids.items()}
            param_combinations.append(config)

    for params in param_combinations:
        logging.info(f"Testing configuration: {params}")
        try:
            # Instantiate the model with parameters (ensure bayesian_model_class accepts these as keyword args)
            model = bayesian_model_class(num_classes=4, pretrained=True, **params).to(device)
            optimizer = torch.optim.Adam(model.parameters(), lr=params.get('lr', 1e-3))
            criterion = torch.nn.CrossEntropyLoss()
            # train_model should be defined elsewhere; here we assume it returns (trained_model, val_loss)
            trained_model, val_loss = train_model(model, train_loader, val_loader, epochs=params.get('epochs', 5),
                                                  bayesian=True, class_weighted_loss=class_weighted_loss)
            results.append({'config': params, 'val_loss': val_loss})
            logging.info(f"Configuration {params} resulted in val_loss: {val_loss:.4f}")
        except Exception as e:
            logging.error(f"Error with configuration {params}: {e}")

    # Save results
    df_results = pd.DataFrame(results)
    df_results.to_csv(f"{output_dir}/tuning_results_{bayesian_model_class.__name__}.csv", index=False)
    logging.info("Hyperparameter tuning complete.")
    best_config = min(results, key=lambda x: x['val_loss'])
    logging.info(f"Best configuration: {best_config}")
    return best_config
